In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(2).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 14:59:03.791560: {'loss': 2.2424725415229796, 'accuracy': 0.4693375, 'val_loss': 1.5933529810540994, 'val_accuracy': 0.5259}
Epoch 1 ends at 15:01:00.132993: {'loss': 1.6416354302883147, 'accuracy': 0.6219, 'val_loss': 2.048162074605371, 'val_accuracy': 0.475}
Epoch 2 ends at 15:03:01.021586: {'loss': 1.1950815812826157, 'accuracy': 0.683, 'val_loss': 1.0784382744199912, 'val_accuracy': 0.7088}
Epoch 3 ends at 15:05:03.332844: {'loss': 0.8296974552631378, 'accuracy': 0.7527375, 'val_loss': 0.854839674416621, 'val_accuracy': 0.727}
Epoch 4 ends at 15:07:06.897081: {'loss': 0.5881182085990906, 'accuracy': 0.807225, 'val_loss': 0.9539957843768369, 'val_accuracy': 0.7024}
Epoch 5 ends at 15:09:15.271034: {'loss': 0.4238888278543949, 'accuracy': 0.8546125, 'val_loss': 0.7556287970892184, 'val_accuracy': 0.7665}
Epoch 6 ends at 15:11:26.228374: {'loss': 0.31840539776086807, 'accuracy': 0.8894875, 'val_loss': 0.7997205697806777, 'val_accuracy': 0.7675}
Epoch 7 ends at 15:13:35

In [11]:
model.evaluate(test_data)

157/157 [==============================].4107 - accuracy: 0.87 - 0s 62ms/step - loss: 0.7575 - accuracy: 0.85 - 0s 59ms/step - loss: 0.9407 - accuracy: 0.83 - 0s 56ms/step - loss: 0.9192 - accuracy: 0.85 - 0s 54ms/step - loss: 0.8695 - accuracy: 0.85 - 0s 53ms/step - loss: 0.8296 - accuracy: 0.85 - 0s 51ms/step - loss: 0.8344 - accuracy: 0.85 - 0s 49ms/step - loss: 0.8747 - accuracy: 0.84 - 0s 48ms/step - loss: 0.8661 - accuracy: 0.83 - 0s 48ms/step - loss: 0.8709 - accuracy: 0.83 - 1s 48ms/step - loss: 0.8356 - accuracy: 0.83 - 1s 48ms/step - loss: 0.8566 - accuracy: 0.82 - 1s 47ms/step - loss: 0.8879 - accuracy: 0.81 - 1s 48ms/step - loss: 0.8783 - accuracy: 0.81 - 1s 48ms/step - loss: 0.8705 - accuracy: 0.81 - 1s 48ms/step - loss: 0.8979 - accuracy: 0.81 - 1s 47ms/step - loss: 0.8942 - accuracy: 0.81 - 1s 47ms/step - loss: 0.9330 - accuracy: 0.80 - 1s 47ms/step - loss: 0.9085 - accuracy: 0.81 - 1s 46ms/step - loss: 0.9175 - accuracy: 0.81 - 1s 46ms/step - loss: 0.9083 - accuracy: 0.

[1.0497843628856027, 0.8001]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))